In [1]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# clone code repository
!git clone https://github.com/project-delphi/spider-schema-gnn-global.git
#!git clone https://github.com/benbogin/spider-schema-gnn-global.git
#!pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp37-cp37m-linux_x86_64.whl # CUDA 10.0 build

Cloning into 'spider-schema-gnn-global'...
remote: Enumerating objects: 130, done.
remote: Counting objects: 100% (130/130), done.
remote: Compressing objects: 100% (77/77), done.
remote: Total 130 (delta 51), reused 117 (delta 44), pack-reused 0
Receiving objects: 100% (130/130), 79.55 KiB | 445.00 KiB/s, done.
Resolving deltas: 100% (51/51), done.


In [3]:
!pip install --upgrade torch-scatter
!pip install --upgrade torch-sparse
!pip install --upgrade torch-cluster
!pip install --upgrade torch-spline-conv 
!pip install torch-geometric

  Created wheel for torch-scatter: filename=torch_scatter-1.4.0-cp36-cp36m-linux_x86_64.whl size=2875369 sha256=3786fe33f062590dbc12d7b137346b83b65209e6ecc8bd9794cc514ac6d9cafc
  Stored in directory: /root/.cache/pip/wheels/25/00/c4/1637b4b3003f29092f4fe2ad4b40dd10906269c1ac2dc82941
Successfully built torch-scatter
  Created wheel for torch-sparse: filename=torch_sparse-0.4.3-cp36-cp36m-linux_x86_64.whl size=3966434 sha256=5592918876516cc241bd9a950a6923d18ebc690d068a4d657a27c96ef06af661
  Stored in directory: /root/.cache/pip/wheels/02/66/2b/befece01c2516f9fb3e7b4d150bb2b871221c73657c9cd7735
Successfully built torch-sparse
  Created wheel for torch-cluster: filename=torch_cluster-1.4.5-cp36-cp36m-linux_x86_64.whl size=16219409 sha256=9e01dae5cc22555a586133a72e71017c2beb787df6343c4d91d3b0b374eda372
  Stored in directory: /root/.cache/pip/wheels/0a/26/7e/a6d6a80eae5ca39b92bc77773f36cf433d5085de18014382b1
Successfully built torch-cluster
  Created wheel for torch-spline-conv: filename=tor

In [4]:
# python model dependencies
%cd spider-schema-gnn-global/
!pip install -r requirements.txt
!python -c "import nltk; nltk.download('punkt')"

/content/spider-schema-gnn-global
     |████████████████████████████████| 37.4MB 161kB/s 
     |████████████████████████████████| 25.2MB 56.5MB/s 
     |████████████████████████████████| 5.6MB 32.3MB/s 
     |████████████████████████████████| 1.9MB 30.2MB/s 
     |████████████████████████████████| 614kB 44.1MB/s 
     |████████████████████████████████| 194kB 41.1MB/s 
     |████████████████████████████████| 133kB 43.2MB/s 
     |████████████████████████████████| 512kB 39.7MB/s 
     |████████████████████████████████| 133kB 35.1MB/s 
     |████████████████████████████████| 92kB 11.8MB/s 
     |████████████████████████████████| 4.5MB 39.1MB/s 
     |████████████████████████████████| 552kB 30.9MB/s 
     |████████████████████████████████| 2.5MB 31.4MB/s 
     |████████████████████████████████| 12.6MB 29.1MB/s 
     |████████████████████████████████| 245kB 47.5MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
     |████████████████████████████████| 61kB 8.9MB/s 
     |███████████

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [5]:
# mount resources
# spider dataset
!mkdir /content/spider-schema-gnn-global/dataset
!cp -r /content/drive/My\ Drive/models/nlp/spider/* /content/spider-schema-gnn-global/dataset
# pretrained model
!mkdir -p /content/spider-schema-gnn-global/experiments
!tar xvzf /content/drive/My\ Drive/models/nlp/allen-gnn/gnn_global_models.tar.gz -C /content/spider-schema-gnn-global/experiments

experiment/
experiment/config.json
experiment/best.th
experiment/vocabulary/
experiment/vocabulary/tokens.txt
experiment/vocabulary/rule_labels.txt
experiment/vocabulary/non_padded_namespaces.txt
experiment/candidates_dev.json
experiment/candidates_train.json
experiment_rerank/
experiment_rerank/config.json
experiment_rerank/best.th
experiment_rerank/vocabulary/
experiment_rerank/vocabulary/tokens.txt
experiment_rerank/vocabulary/non_padded_namespaces.txt


In [6]:
!allennlp predict experiments/experiment dataset/dev.json \
--predictor spider_predict_complete \
--use-dataset-reader \
--cuda-device=0 \
--output-file output.sql \
--silent \
--include-package models.semantic_parsing.spider_parser \
--include-package dataset_readers.spider \
--include-package predictors.spider_predictor_complete \
--weights-file experiments/experiment/best.th \
-o "{\"dataset_reader\":{\"keep_if_unparsable\":true}}"

2019-11-14 04:03:30,781 - INFO - pytorch_pretrained_bert.modeling - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
/usr/local/lib/python3.6/dist-packages/sklearn/utils/linear_assignment_.py:21: DeprecationWarning: The linear_assignment_ module is deprecated in 0.21 and will be removed from 0.23. Use scipy.optimize.linear_sum_assignment instead.
  DeprecationWarning)
2019-11-14 04:03:32,819 - INFO - allennlp.models.archival - loading archive file experiments/experiment
2019-11-14 04:03:32,843 - INFO - allennlp.common.params - type = default
2019-11-14 04:03:32,844 - INFO - allennlp.data.vocabulary - Loading token dictionary from experiments/experiment/vocabulary.
2019-11-14 04:03:32,848 - INFO - allennlp.common.from_params - instantiating class <class 'allennlp.models.model.Model'> from params {'action_embedding_dim': 200, 'dataset_path': 'dataset/', 'decoder_beam_search': {'beam_size': 10}, 'decoder_self_attend': True, 'decoder_use_graph_enti

In [0]:
def sem2sql(data):
  with open('output.sql', 'r') as file:
  """
    create query object (nl, table schema)
    create subprocess from data and allennlp
     command
  """
  sql_string = 'Coming soon, the greatest sqlizer'
  return sql_string

In [11]:
# ngrok
%cd /content
!cp /content/drive/My\ Drive/assets/ngrok/ngrok-stable-linux-amd64.zip /content
!unzip /content/ngrok-stable-linux-amd64.zip
# ngrok secret a bit hacky, first setup with temp token
!/content/ngrok authtoken temp
# now replace token
!cp /content/drive/My\ Drive/assets/ngrok/ngrok.yml /root/.ngrok2
%cd /content/spider-schema-gnn-global/


/content
Archive:  /content/ngrok-stable-linux-amd64.zip
replace ngrok? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
Authtoken saved to configuration file: /root/.ngrok2/ngrok.yml
/content/spider-schema-gnn-global


In [12]:
#flask
!pip install flask
#flask-ngrok
!pip install flask-ngrok


In [0]:
from flask import Flask, request, jsonify
from flask_ngrok import run_with_ngrok

app = Flask(__name__)
run_with_ngrok(app)   #starts ngrok when the app is run

@app.route("/")
def home():
    return "<h1> Running Flask on Google Colab, isn't this cool! </h1>"
  

@app.route('/query', methods=['POST']) 
def query():
    if request.method == "POST":
        features = request.form
    sql = sem2sql(features)
    return jsonify(sql), 200

app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
Exception in thread Thread-7:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 1182, in run
    self.function(*self.args, **self.kwargs)
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 70, in start_ngrok
    ngrok_address = _run_ngrok()
  File "/usr/local/lib/python3.6/dist-packages/flask_ngrok.py", line 38, in _run_ngrok
    tunnel_url = j['tunnels'][0]['public_url']  # Do the parsing of the get
IndexError: list index out of range

